In [56]:
import pandas as pd
import numpy as np
import wrangle as w

pd.set_option('display.max_rows', 500)

In [2]:
df = w.acquire_anonymized_curriculum_access_data()

In [3]:
df

,date,time,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2.0
...,...,...,...,...,...,...,...,...,...,...
900218,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0
900219,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0
900220,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0
900221,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,Staff,2014-02-04,2014-02-04,2.0


In [4]:
# 1 - PHP
# 2 - Java
# 3 - Data Science
# 4 - Front End

In [5]:
df = w.clean_the_data(df)

In [6]:
df 

,date,time,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
date,,,,,,,,,,
2018-01-26,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,PHP
2018-01-26,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,PHP
2018-01-26,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,PHP
2018-01-26,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,PHP
2018-01-26,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,Java
...,...,...,...,...,...,...,...,...,...,...
2021-04-21,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,Java
2021-04-21,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,Java
2021-04-21,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,Java


## 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 to 2021-04-21
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        900223 non-null  datetime64[ns]
 1   time        900223 non-null  object        
 2   page        900222 non-null  object        
 3   user_id     900223 non-null  int64         
 4   cohort_id   847330 non-null  float64       
 5   ip          900223 non-null  object        
 6   name        847330 non-null  object        
 7   start_date  847330 non-null  datetime64[ns]
 8   end_date    847330 non-null  datetime64[ns]
 9   program_id  900223 non-null  object        
dtypes: datetime64[ns](3), float64(1), int64(1), object(5)
memory usage: 75.5+ MB


In [11]:
#change df name!
active_class_access = df.query('start_date < date < end_date')

In [37]:
#278 is weird because they only started accessing the curriculum when the cohort was almost over
#Those who started accessing the curriculum when the cohort was almost over
#Java students
#278, 812
active_class_access[active_class_access.user_id == 278]

,date,time,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
date,,,,,,,,,,
2018-09-27,2018-09-27,13:57:44,/,278,24.0,97.105.19.58,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-27,2018-09-27,14:47:37,java-ii/arrays,278,24.0,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-27,2018-09-27,14:58:48,java-ii/arrays,278,24.0,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-27,2018-09-27,14:59:07,java-ii/collections,278,24.0,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,Java


In [66]:
active_class_access[active_class_access.user_id == 971]

,date,time,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
date,,,,,,,,,,
2021-04-13,2021-04-13,16:10:31,appendix/git/intellij-intro,971,139.0,136.50.89.186,Oberon,2021-04-12,2021-10-01,Java
2021-04-15,2021-04-15,14:50:27,/,971,139.0,136.50.89.186,Oberon,2021-04-12,2021-10-01,Java
2021-04-15,2021-04-15,14:50:32,toc,971,139.0,136.50.89.186,Oberon,2021-04-12,2021-10-01,Java
2021-04-15,2021-04-15,14:50:37,javascript-i/introduction/primitive-types,971,139.0,136.50.89.186,Oberon,2021-04-12,2021-10-01,Java
2021-04-15,2021-04-15,14:58:43,javascript-i/introduction/operators,971,139.0,136.50.89.186,Oberon,2021-04-12,2021-10-01,Java
2021-04-15,2021-04-15,14:59:13,javascript-i/introduction/primitive-types,971,139.0,136.50.89.186,Oberon,2021-04-12,2021-10-01,Java
2021-04-15,2021-04-15,15:02:43,javascript-i/introduction/primitive-types,971,139.0,136.50.89.186,Oberon,2021-04-12,2021-10-01,Java
2021-04-15,2021-04-15,15:02:47,javascript-i/introduction/operators,971,139.0,136.50.89.186,Oberon,2021-04-12,2021-10-01,Java
2021-04-15,2021-04-15,15:09:35,javascript-i/introduction/primitive-types,971,139.0,136.50.89.186,Oberon,2021-04-12,2021-10-01,Java


In [ ]:
#24 seems to have completed the course but barely looked at the curriculum

#41 only started accessing the curriculum when the cohort was almost finished

In [ ]:
#The below users accessed the curriculum on the first day (or first week) and then stopped. I assume these are students who dropped out.
#Java
#879, 832,  388, 572, 772, 529, Oberon Cohort(968, 963, 956, 961, 972, 973, 967, 959, 971)

#Data Science
#679, 

#487 only accessed the appendix pages one day during the first month of classes and that's it. Also, data science

In [32]:
usage = active_class_access.user_id.value_counts().sort_values()

In [33]:
usage[usage <100]

879     1
679     3
832     3
278     4
956     5
812     7
388     8
572    12
889    16
968    16
487    17
963    17
961    18
772    20
49     25
64     25
976    25
24     26
41     28
972    31
973    32
967    33
529    35
959    38
971    41
964    42
371    42
981    42
960    45
741    47
978    49
965    51
56     52
962    52
543    53
46     54
57     62
969    62
858    63
45     65
36     65
975    66
250    67
65     73
52     75
977    76
974    76
13     79
33     79
955    80
702    85
806    88
954    88
253    89
839    92
55     95
958    95
734    96
Name: user_id, dtype: int64

## 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?